## Assignment2 - Supervised Learning flow

### Part 1 - Student details:
* Please write the First Name and last 4 digits of the i.d. for each student. For example:
<pre>Israel 9812</pre>

In [ ]:
# student 1:Yonatan 3212
# student 2:Mor 3239
# student 3:Yevgenia 3208
# student 4:Adi 3186
# student 5:Sahar 2091
# (optional) student 6:

## Part 2 - Experiments
You could add as many code cells as needed

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, make_scorer

In [ ]:

train_file_path = 'wine_train.csv'
test_file_path = 'wine_test.csv'

train_df = pd.read_csv(train_file_path)
test_df = pd.read_csv(test_file_path)

print("Train Set:")
print(train_df.head())

print("\nTest Set:")
print(test_df.head())


In [ ]:
train_stats = train_df.describe()
print("Train Data Statistics:",train_stats)          # חישוב סטטיסטיקות עבור המשתנים המספריים בנתוני האימון

corr_matrix = train_df.corr() 
print("\nCorrelation Matrix:\n",corr_matrix)         # חישוב מטריצת הקורלציה של נתוני האימון

print("\nTrain Data Info:")                          # הדפסת מידע על נתוני האימון
print(train_df.info())

print("\nTest Data Info:")                           # הדפסת מידע כללי על נתוני הבדיקה
print(test_df.info())

plt.figure(figsize=(5, 3))
plt.scatter(train_df['alcohol'], train_df['proline'], alpha=0.7, s=100, color='Indianred')
plt.title('Alcohol vs Proline')
plt.xlabel('Alcohol')
plt.ylabel('Proline')
plt.grid(True)
plt.show()                                           #עבור אחוז אלכוהול והפרולין scatterplot יצירת דיאגרמת 

plt.figure(figsize=(5, 3))
plt.hist(train_df['alcohol'], bins=15, color='Darkred', edgecolor='black')
plt.title('Distribution of Alcohol')
plt.xlabel('Alcohol')
plt.ylabel('Frequency')
plt.show()                                            # יצירת היסטוגרמה עבור שכיחות אחוזי האלכהול

In [ ]:
train_labels = train_df['target']       # שמירת תווית התוצאה מהמאפיינים בנתוני האימון ובנתוני הבדיקה והוצאתן מהדאטאסט
test_labels = test_df['target']
train_features = train_df.drop(columns=['target'])
test_features = test_df.drop(columns=['target'])

numeric_features = train_features.select_dtypes(include=[np.number]).columns.tolist()    #סינון המאפיינים המספריים 
scaler = StandardScaler()  # יצירת מודל סקאלינג לנירמול בכדי להבטיח שכל הנתונים יהיו מנורמלים
train_features[numeric_features] = scaler.fit_transform(train_features[numeric_features])  # תהליך נירמול על נתוני האימון
test_features[numeric_features] = scaler.transform(test_features[numeric_features])  # תהליך נירמול על נתוני הבדיקה
                                                                                
categorical_features = train_features.select_dtypes(include=['object']).columns.tolist()
if categorical_features:                 # סינון המאפיינים הקטגוריים
    encoder = OneHotEncoder(sparse=False, drop='first')  # נשנה את כל המאפיינים הקטגורים למאפיינים מספריים 
    encoded_train = pd.DataFrame(encoder.fit_transform(train_features[categorical_features]),
                                 columns=encoder.get_feature_names_out(categorical_features))  
    encoded_test = pd.DataFrame(encoder.transform(test_features[categorical_features]),
                                columns=encoder.get_feature_names_out(categorical_features)) 
    
    train_features = pd.concat([train_features, encoded_train], axis=1)  #נכניס את המשתנים הקטגורים המקודדים
    test_features = pd.concat([test_features, encoded_test], axis=1)     # לדאטא סטים כעמודות

    train_features.drop(columns=categorical_features, inplace=True)  # נסיר את המאפיינים הקטגורים מהדאטאסט אימון ובדיקה
    test_features.drop(columns=categorical_features, inplace=True) 

poly = PolynomialFeatures(degree=2, include_bias=False)
poly_train_features = poly.fit_transform(train_features)  # יצירת מאפיינים פולינומיאליים עבור נתוני האימון והבדיקה
poly_test_features = poly.transform(test_features)       # בכדי לשפר את יכולת המודל לזהות יחסים לא לינאריים

X_train = poly_train_features
X_test = poly_test_features
y_train = train_labels
y_test = test_labels


In [ ]:
knn_model = KNeighborsClassifier()               # KNN יצירת אובייקט סיווג
param_grid_knn = {
    'n_neighbors': [3, 5, 7],                 # הגדרת הפרמטרים לבדיקה הכולל מספר קרובים אפשריים ופונקציית המשקל
    'weights': ['uniform', 'distance'],
}
grid_search_knn = GridSearchCV(estimator=knn_model, param_grid=param_grid_knn, cv=5, scoring='accuracy')
grid_search_knn.fit(X_train, y_train)            # KNN לחיפוש פרמטרים אופטימאלים עבור סיווג GridSearchCV יצירת אובייקט

print("Best Parameters for KNN:", grid_search_knn.best_params_)

In [ ]:
tree_model = DecisionTreeClassifier()         # DecisionTree יצירת אובייקט סיווג
param_grid_tree = {
    'max_depth': [None, 10, 20, 30],          # הגדרת הפרמטרים לבדיקה הכוללעומק מקסימלי של העץ ופיצול צמתים מינימאלי
    'min_samples_split': [2, 10, 20]
}
grid_search_tree = GridSearchCV(estimator=tree_model, param_grid=param_grid_tree, cv=5, scoring='accuracy')
grid_search_tree.fit(X_train, y_train)    # DecisionTree לחיפוש פרמטרים אופטימאלים עבור סיווג GridSearchCV יצירת אובייקט

best_params_tree = grid_search_tree.best_params_

print("Best Parameters for the Decision Tree:", best_params_tree)

In [ ]:
macro_f1_scorer = make_scorer(f1_score, average='macro')        # f1_scores יצירת הערכת ביצועים למודלים לפי שיטת ממוצע  

model_options = ['KNN', 'Decision Tree']    #בדיקת שני הסיווגים לפי כמות  קירובים אפשריים ופונקציית המשקל
                                            # או במקרה של עץ ההחלטה לפי עומק מקסימלי של העץ וכמות פיצולים מינימאלית
hyperparameters = { 
    'KNN': {'n_neighbors': [3, 5, 7, 9], 'weights': ['uniform', 'distance']},
    'Decision Tree': {'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 10, 20]}
}

results_list = []

for model_option in model_options:                           #יצירת לולאה אשר תבדוק את כל המקרים של כל סיווג
    if model_option == 'KNN':
        model = KNeighborsClassifier()
    elif model_option == 'Decision Tree':
        model = DecisionTreeClassifier()
        
    param_grid = hyperparameters[model_option]
    
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring=macro_f1_scorer)
    grid_search.fit(X_train, y_train)   #לחיפוש פרמטרים אופטימאלים עבור הסיווג הנבדק ברגע זה GridSearchCV יצירת אובייקט
                                        #5 fold cross validation מסמל את שיטת ה cv=5
    
   
    best_score = grid_search.best_score_                     # קבלת התוצאות הטובות ביותר והוספתם לרשימת התוצאות
    best_params = grid_search.best_params_
    results_list.append({'Model': model_option,
                         'Hyperparameters': best_params,
                         'Macro-average-f1': best_score})

results_df = pd.DataFrame(results_list)

print(results_df)

## Part 3 - Training flow
Use the best combination of feature engineering, model (algorithm and hyperparameters) from the experiment part (part 2)

In [ ]:
best_combination = results_df.loc[results_df['Macro-average-f1'].idxmax()]    #בחירת השורה הטובה ביותר מההשוואה

if best_combination['Model'] == 'KNN':                #אז ניצור אובייקט עם הנתונים שלו KNN במידה והסיווג היותר טוב הוא
    model = KNeighborsClassifier(**best_combination['Hyperparameters'])         
elif best_combination['Model'] == 'Decision Tree':    #אז ניצור אובייקט עם הנתונים שלו Decision Tree במידה והסיווג היותר טוב הוא
    model = DecisionTreeClassifier(**best_combination['Hyperparameters'])

model.fit(X_train, y_train)       # נאמן את המודל עם נתוני האימון



## Part 4 - Apply on test and show model performance estimation

In [ ]:
test_predictions = model.predict(X_test)         #חיזוי נתוני הבדיקה עם המודל שאימנו

test_accuracy = accuracy_score(y_test, test_predictions)        # חישוב דיוק החיזוי 
test_f1_macro = f1_score(y_test, test_predictions, average='macro') #עבור נתוני הבדיקה f1-score חישוב ה

print("Test Accuracy:", test_accuracy)
print("Test Macro-average F1 Score:", test_f1_macro)